# Grammar Correction using LLM

In [3]:
import torch
# import optuna
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from nltk.translate.bleu_score import sentence_bleu
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import SmoothingFunction
from transformers import T5Tokenizer, T5ForConditionalGeneration

## Dataset

In [17]:
# Grammar Correction Dataset : https://www.kaggle.com/datasets/satishgunjal/grammar-correction

df = pd.read_csv("./dataset/Grammar Correction.csv", sep=",")
df.head()

,Serial Number,Error Type,Ungrammatical Statement,Standard English
0,1,Verb Tense Errors,I goes to the store everyday.,I go to the store everyday.
1,2,Verb Tense Errors,They was playing soccer last night.,They were playing soccer last night.
2,3,Verb Tense Errors,She have completed her homework.,She has completed her homework.
3,4,Verb Tense Errors,He don't know the answer.,He doesn't know the answer.
4,5,Verb Tense Errors,The sun rise in the east.,The sun rises in the east.


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018 entries, 0 to 2017
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Serial Number            2018 non-null   int64 
 1   Error Type               2018 non-null   object
 2   Ungrammatical Statement  2018 non-null   object
 3   Standard English         2018 non-null   object
dtypes: int64(1), object(3)
memory usage: 63.2+ KB


In [12]:
# count of error types

error_counts = df['Error Type'].value_counts()
print(error_counts)

Error Type
Sentence Structure Errors                         103
Verb Tense Errors                                 100
Subject-Verb Agreement                            100
Article Usage                                     100
Spelling Mistakes                                 100
Preposition Usage                                  95
Punctuation Errors                                 60
Relative Clause Errors                             51
Gerund and Participle Errors                       50
Abbreviation Errors                                50
Slang, Jargon, and Colloquialisms                  50
Negation Errors                                    50
Incorrect Auxiliaries                              50
Ambiguity                                          50
Tautology                                          50
Lack of Parallelism in Lists or Series             50
Mixed Metaphors/Idioms                             50
Parallelism Errors                                 49
Contractions Erro